**Importing packages that we need**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, njit, vectorize

**Setting up the Monte Carlo method for a NxN lattice:**

In [11]:
#function that defines that inital state for the lattice calling a random number generator
@jit
def StateInitalize(N):
    #NXN number of spins
    Lattice = np.zeros((N,N))
    for i in range(N):
        for l in range(N):
            randNum = np.random.random()
            if randNum < 0.5:
                Lattice[i,l] = int(-1)
            else:
                Lattice[i,l] = int(1)
                
    return Lattice

In [29]:
A = StateInitalize(4)
print(A)

[[ 1. -1. -1. -1.]
 [ 1. -1. -1.  1.]
 [ 1. -1. -1. -1.]
 [-1. -1.  1. -1.]]


In [31]:
def MonteCarloIsing(InitialState,NumOfCycles,LatticeSize):
    M = LatticeSize
    MC = NumOfCycles
    for i in range(MC):#Looping over the number of MC cycles
        #picking a random coordinate in the lattice
        randNum = np.random.randint(M*M)
        line = int(np.floor(randNum/M))#finding the relavant line in the lattice
        column = int(randNum - M*line)#finding the relavant column in the lattice
        print(randNum,line,column)
        #flipping the spin at that position
        if InitialState[line,column] == 1:
            InitialState[line,column] = int(-1)
        else:
            InitialState[line,column] = int(1)
        #finding the difference in energy - using periodic boundary conditions
        if line == 0:#if at the top edge of lattice
            Temp1 = InitialState[line,column]*InitialState[line,column-1]#to the left
            Temp2 = InitialState[line,column]*InitialState[line,column+1]#to the right
            Temp3 = InitialState[line,column]*InitialState[line+1,column]#above
            Temp4 = InitialState[line,column]*InitialState[line+1,column]#below
        if line == (M-1):#if at the bottom edge lattice
            Temp1 = InitialState[line,column]*InitialState[line,column-1]#to the left
            Temp2 = InitialState[line,column]*InitialState[line,column+1]#to the right
            Temp3 = InitialState[line,column]*InitialState[line-1,column]#above
            Temp4 = InitialState[line,column]*InitialState[line-1,column]#below
        if column == 0:#if at the left edge
            Temp1 = InitialState[line,column]*InitialState[line,column+1]#to the left
            Temp2 = InitialState[line,column]*InitialState[line,column+1]#to the right
            Temp3 = InitialState[line,column]*InitialState[line+1,column]#above
            Temp4 = InitialState[line,column]*InitialState[line+1,column]#below
        if column == (M-1):#if at the right edge
            Temp1 = InitialState[line,column]*InitialState[line,column-1]#to the left
            Temp2 = InitialState[line,column]*InitialState[line,column-1]#to the right
            Temp3 = InitialState[line,column]*InitialState[line+1,column]#above
            Temp4 = InitialState[line,column]*InitialState[line+1,column]#below
        else:
            Temp1 = InitialState[line,column]*InitialState[line,column-1]#to the left
            Temp2 = InitialState[line,column]*InitialState[line,column+1]#to the right
            Temp3 = InitialState[line,column]*InitialState[line+1,column]#above
            Temp4 = InitialState[line,column]*InitialState[line-1,column]#below
            
        DeltaE = -(Temp1 + Temp2 + Temp3 + Temp4)
        
        print(DeltaE)
        

In [32]:
MonteCarloIsing(A,10,4)

11 2 3
4.0
3 0 3


IndexError: index 4 is out of bounds for axis 1 with size 4